# Import Libraries

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector

from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv
load_dotenv()


True

# Loading The LLM (Language Model)


In [2]:
llm = Ollama(model="nuextract", base_url="http://127.0.0.1:11434")

C:\Users\Predator\AppData\Local\Temp\ipykernel_31992\570180220.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="nuextract", base_url="http://127.0.0.1:11434")


# Setting OpenAI Embeddings

In [3]:
embed_model = OllamaEmbeddings(
    model="nuextract",
    base_url='http://127.0.0.1:11434'
)

C:\Users\Predator\AppData\Local\Temp\ipykernel_31992\1909610071.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embed_model = OllamaEmbeddings(


# Loading HTML Content

In [4]:
loader = TextLoader("sample.html",encoding='utf-8')
documents = loader.load()

In [ ]:
documents

# Splitting Text Into Chunks

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=int(1e6), chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# PgAI Stuff

In [7]:
embeddings = embed_model
connection_string = os.environ.get('POSTGRES_CONNECTION_STRING')
collection_name = "monkey"
vector_store = PGVector.from_documents(
    embedding=embeddings,
    collection_name=collection_name,
    connection_string=os.environ.get('POSTGRES_CONNECTION_STRING'),
    use_jsonb=True,
    documents=docs
)

# Creating a Retriever


In [8]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 100})

# Creating a Retrieval Chain


In [9]:
chain = create_retrieval_chain(combine_docs_chain=llm,retriever=retriever)

#  Retrieval-QA Chat Prompt


In [ ]:
retrieval_qa_chat_prompt = PromptTemplate(
    input_variables=["context", "input"],
    template = '''
        %INSTRUCTIONS:
        You are a professional web scraper specializing in extracting HTML elements and their content based on the specified context.

        Please extract only the requested HTML tags and their contents according to the given context. Respond strictly in the following format:
        <tag_name>content of the tag</tag_name>
        ...

        Do not include any additional text, explanations, or formatting outside of the specified structure.

        %CONTEXT: 
        The type of tags or content to extract is as follows:
        {context}

        %PROMPT: 
        extract all of the following html tag and there content
        {input}
    '''
)


# Combining Documents


In [20]:
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

# Final Retrieval Chain


In [21]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)    

# Invoking the Retrieval Chain


In [26]:
response = retrieval_chain.invoke({"input": "p"}) # Please enter the HTML tag you want to retrieve 

In [29]:
response['answer']

' <p>Monkeys are primates found in various parts of the world. Known for their intelligence and playful behavior, they are an integral part of tropical and subtropical ecosystems.</p>\n<p>There are over 260 species of monkeys, divided into two main groups: Old World monkeys, found primarily in Africa and Asia, and New World monkeys, found in Central and South America.</p>\n<p>Monkeys thrive in diverse habitats ranging from rainforests to savannas. Most monkeys live in trees, although some, like baboons, spend more time on the ground.</p>\n<p>Here are some interesting facts about monkeys:</p>\n<p>Monkeys use tools like sticks to extract insects from tree bark.</p>\n<p>Some species are known to form complex social groups and communicate using various vocalizations.</p>\n<p>Monkeys have an excellent memory, which helps them locate food and recognize individuals within their group.</p>\n<p>&copy; 2024 Monkey Information Center</p>'